In [1]:
#imports and set up

from __future__ import division, unicode_literals, print_function  # for compatibility with Python 2 and 3
from IPython.display import display
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import time
import sys
from process_pickle import process_pickle
from process_pickle_quiver import process_pickle_quiver
%matplotlib inline
from ipywidgets import interact, interactive, fixed #Sliders for image selection
import ipywidgets as widgets
mpl.rc('figure',  figsize=(4.77, 2.95))
mpl.rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
mpl.rcParams['lines.linewidth'] = 0.6
mpl.rcParams['lines.color'] = 'r'
plt.rc('grid', linestyle="--", color='gray')

import numpy as np
import pandas as pd
from pandas import DataFrame, Series  

import pims
import trackpy as tp
import os

In [2]:
# function that is called to configure how plots are to be drawn

def setup_plot():
    ax = plt.gca()
    SPINE_COLOR = 'gray'
    for spine in ['top', 'right']:
            ax.spines[spine].set_visible(False)

    for spine in ['left', 'bottom']:
        ax.spines[spine].set_color(SPINE_COLOR)
        ax.spines[spine].set_linewidth(0.5)

    ax.xaxis.set_ticks_position('bottom')
    ax.yaxis.set_ticks_position('left')

    for axis in [ax.xaxis, ax.yaxis]:
        axis.set_tick_params(direction='out', color=SPINE_COLOR)
    plt.grid()
    fig = plt.gcf()
    fig.set_size_inches(4.77, 2.95, forward=True)

Use the file picker to locate the .pkl containing the particle trajectories

or select a previously output .xlsx to skip the calculations

**If it doesn't appear you probably need to minimise your browser or alt tab around for it**

In [3]:
set_custom_data_directory = True

In [4]:
# select the .pkl files you wish to analyse. When you're done selecting cancel should
# allow you to continue to the next cell.

import tkinter as tk
from tkinter import filedialog

pkl_to_process = []
file_path = "1"
while len(file_path) != 0:
    root = tk.Tk()
    root.withdraw()
    file_path = filedialog.askopenfilename()
    if len(file_path) != 0:
        pkl_to_process.append(file_path)

In [5]:
# confirm you've selected the correct files

pkl_to_process

['/Users/liamchalcroft/Python/25C 1_8gL data first 1000 frames/trajectories.pkl']

In [6]:
calculate = True

In [7]:
# configure the calculations to be carried out on the data contained in the .pkl files
# if you don't enter "NEXT" at the end you can queue another set of calculations to be carried out on the 
# same .pkl, e.g with a frame lookback.
pkl_param = []

i = 0
mpp = 15 # microns per pixel
timestep = 30 # seconds between images

while i < len(pkl_to_process):
    print(pkl_to_process[i])
    print("Setting Parameters...")
    # uncomment the below lines to set mpp or timestep on a per .pkl basis
    #mpp = int(input("Microns per Pixel: "))
    #timestep = int(input("Time between frames in seconds: "))
    frame_lookback = int(input("How many frames to look back for velocity calculations? "))
    pkl_param.append([i,mpp,timestep,frame_lookback])
    move_next = input("Enter NEXT to move to next pkl, anything else to add another analysis run for the current particle")
    if move_next == "NEXT":
        i += 1
    

/Users/liamchalcroft/Python/25C 1_8gL data first 1000 frames/trajectories.pkl
Setting Parameters...


How many frames to look back for velocity calculations?  25
Enter NEXT to move to next pkl, anything else to add another analysis run for the current particle NEXT


In [8]:
pkl_param

[[0, 15, 30, 25]]

In [9]:
# for the sets of calculations you've requested, select folders to store the results for each set in.

data_dirs = []
data_dir = True
counter = 0
for i in pkl_param:
    root = tk.Tk()
    root.withdraw()
    data_dir = filedialog.askdirectory(title="Pick data dir for {} {}".format(pkl_to_process[i[0]], i[3]))
    data_dirs.append(data_dir)

In [10]:
# if the folders you've requested the results stored in don't exist, make them:

for directory in data_dirs:
    if not os.path.exists(directory):
        os.makedirs(directory)

In [12]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
warnings.filterwarnings("ignore",category=FutureWarning)

for c, param in enumerate(pkl_param):
    # the process_pickle_quiver function is contained within a separate file - process_pickle_quiver.py
    df = process_pickle_quiver(pkl_to_process[param[0]], data_dirs[c], param[1], param[2], param[3])

 Calculating Displacement for particles: 100.00%

 Calculating Velocity for particle: 100.00%

 Calculating Y Velocity for particle: 100.00%

 Calculating X Velocity for particle: 100.00%

Storing data...
Finished


In [ ]:
# function called to draw the charts of the results

def produce_charts(df, general_directory):
    if not os.path.exists(general_directory + "/figures"):
        os.makedirs(general_directory + "/figures")
    no_bins = df.bin.max() + 1    
    av_disp_by_frame = df.groupby('frame').displacement.mean().sort_index()
    xs = av_disp_by_frame.index.values
    ys = av_disp_by_frame.values
    plt.figure()
    setup_plot()
    plt.plot(xs, ys)
    plt.ylabel("Displacement / px")
    plt.xlabel("Frame")
    plt.tight_layout()
    plt.savefig(general_directory + "/figures/Average_displacement.pdf")
    fig = plt.gcf()


    disp_vels = df.groupby('particle').velocity.mean()
    plt.figure()
    setup_plot()
    plt.xlabel("Velocity / ms$^-1$")
    plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
    plt.hist(disp_vels, bins=20)
    plt.savefig(general_directory + "/figures/Velocity Distribution by Displacement.pdf")


    y_vels = df.groupby('particle').y_velocity.mean()
    plt.figure()
    setup_plot()
    plt.xlabel("Velocity / ms$^-1$")
    plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
    plt.hist(y_vels, bins=20)
    plt.savefig(general_directory + "/figures/Velocity Distribution by Y Velocity.pdf")



    av_vel_by_frame = df.groupby('frame').velocity.mean().sort_index()
    xs = av_vel_by_frame.index.values
    ys = av_vel_by_frame.values
    plt.figure()
    setup_plot()
    plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
    plt.autoscale(enable=True, axis='both', tight=True)
    plt.ylabel("Velocity / ms$^-1$")
    plt.xlabel("Frame")
    plt.autoscale(True)
    plt.tight_layout()
    plt.plot(xs, ys,)
    plt.savefig(general_directory + "/figures/Average_Velocity by Displacement.pdf")

    av_vel_by_frame = df.groupby('frame').y_velocity.mean().sort_index()
    xs = av_vel_by_frame.index.values
    ys = av_vel_by_frame.values
    plt.figure()
    setup_plot()
    plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
    plt.autoscale(enable=True, axis='both', tight=True)
    plt.ylabel("Velocity / ms$^-1$")
    plt.xlabel("Frame")
    plt.autoscale(True)
    plt.plot(xs, ys)
    plt.savefig(general_directory + "/figures/Average_Velocity by Y Position.pdf")
    
    av_vel_by_frame = df.groupby('frame').x_velocity.mean().sort_index()
    xs = av_vel_by_frame.index.values
    ys = av_vel_by_frame.values
    plt.figure()
    setup_plot()
    plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
    plt.autoscale(enable=True, axis='both', tight=True)
    plt.ylabel("Velocity / ms$^-1$")
    plt.xlabel("Frame")
    plt.autoscale(True)
    plt.tight_layout()
    plt.plot(xs, ys)
    plt.savefig(general_directory + "/figures/Average_Velocity by X Position.pdf")
    
    av_vel_by_frame = df.groupby('frame').hyp_vel.mean().sort_index()
    xs = av_vel_by_frame.index.values
    ys = av_vel_by_frame.values
    plt.figure()
    setup_plot()
    plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
    plt.autoscale(enable=True, axis='both', tight=True)
    plt.ylabel("Velocity / ms$^-1$")
    plt.xlabel("Frame")
    plt.autoscale(True)
    plt.tight_layout()
    plt.plot(xs, ys)

    plt.savefig(general_directory + "/figures/Average_Velocity.pdf")

    spacings = []
    for x in df.frame.unique():
        w_df = df[df['frame']==x]
        spacings.append(tp.proximity(w_df)['proximity'].mean())
    plt.figure()
    setup_plot()
    plt.ylabel("Distance / px")
    plt.xlabel("Frame")
    plt.plot(spacings)
    plt.savefig(general_directory + "/figures/nearest_neighbour.pdf")

    bin_vels = df.groupby('bin').hyp_vel.mean()
    plt.figure()
    bin_vels_to_plot = []
    for c, v in enumerate(bin_vels):
        bin_vels_to_plot.append((c, v))

    err = df.groupby('bin').hyp_vel.std()
    x = [x[0] for x in bin_vels_to_plot]
    y = [x[1] for x in bin_vels_to_plot]
    plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
    setup_plot()
    plt.ylabel("Velocity / ms$^-1$")
    plt.xlabel("x position (" + str(no_bins) +" equally spaced bins)")
    plt.tight_layout()
    plt.bar(x, y, yerr = err, error_kw=dict(elinewidth=2,ecolor='black'))

    plt.savefig(general_directory + "/figures/x_binned_velocity.pdf")
    
    bin_vels = df.groupby('bin').y_velocity.mean()
    plt.figure()
    bin_vels_to_plot = []
    for c, v in enumerate(bin_vels):
        bin_vels_to_plot.append((c, v))

    err = df.groupby('bin').y_velocity.std()
    x = [x[0] for x in bin_vels_to_plot]
    y = [x[1] for x in bin_vels_to_plot]
    plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
    setup_plot()
    plt.ylabel("Velocity / ms$^-1$")
    plt.xlabel("x position (" + str(no_bins) +" equally spaced bins)")
    plt.tight_layout()
    plt.bar(x, y, yerr = err, error_kw=dict(elinewidth=2,ecolor='black'))

    plt.savefig(general_directory + "/figures/x_binned_y_velocity.pdf")
    
    

    def x_hist(frame):
        w_df = df[df.frame == frame].copy()
        bin_vels = w_df.groupby('bin').hyp_vel.mean()
        plt.figure()
        bin_vels_to_plot = []
        for c, v in enumerate(bin_vels):
            bin_vels_to_plot.append((c, v))
        x = [x[0] for x in bin_vels_to_plot]
        y = [x[1] for x in bin_vels_to_plot]
        plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
        setup_plot()
        plt.ylabel("Velocity / ms$^-1$")
        plt.xlabel("x position (" + str(no_bins) +" equally spaced bins)")
        plt.xlim(0, 20)
        plt.bar(x, y)



    ### Set use_y_vel_only to true to plot only the y component of the particles' velocities

    from mpl_toolkits.mplot3d import Axes3D
    use_y_vel_only = True

    if use_y_vel_only:
        a = df.groupby([(df.frame // 50) * 50 , 'bin']).y_velocity.mean().fillna(0)
    else:
        a = df.groupby([(df.frame // 50) * 50 , 'bin']).hyp_vel.mean().fillna(0)

    xs = []
    ys = []
    zs = []
    xs = [x[1] for x in a.index.values]
    zs = [x[0] for x in a.index.values]

    for x in a:
        ys.append(x * 1*10**6)  



    colours_alt = []
    options = ['r','g', 'b']
    option_index = 0
    prev_a = 99
    for a in xs:
        if a <= prev_a:
            option_index += 1
            if (option_index)  >= len(options):
                option_index = 0
        colours_alt.append(options[option_index])
        prev_a = a  

    r = 0.99
    colours = []
    add = r / len(xs)
    for x in range(len(xs)):
        colours.append((r, 0, 1 -r))
        r -= add

    %matplotlib inline
    from mpl_toolkits.mplot3d import Axes3D
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.bar(xs, ys, zs=zs, zdir = 'y', color = colours)
    ax.set_xlabel('x bin')
    ax.set_zlabel('Average velocity / um s-1')
    ax.set_ylabel('Frame')
    plt.show()
    plt.savefig(general_directory + "/figures/3d_bar_vel.pdf")

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(xs, zs, ys, color = colours)
    ax.set_xlabel('x bin')
    ax.set_zlabel('Average velocity / um s-1')
    ax.set_ylabel('Frame')
    plt.show() 

    ax = df.groupby('frame')['x'].count().plot()
    fig = ax.get_figure()
    fig.savefig(general_directory + "/figures/particles_in_frame.pdf")

for c, param in enumerate(pkl_param):
    # the process_pickle function is contained within a separate file - process_pickle.py
    df = process_pickle(pkl_to_process[param[0]], data_dirs[c], param[1], param[2], param[3])
    print('Producing plots...')
    print('\n')
    produce_charts(df, data_dirs[c])    
    print('\n')
    print('Finished')  

In [11]:
npstep = int(input('How many frames per animation frame? '))

How many frames per animation frame?  50


In [120]:

def df_to_numpy(general_directory):
    xdf = pd.read_csv(general_directory + '/data_quiver.csv', usecols=[9, 10, 18])
    xdf = xdf.reset_index().pivot_table(index='frame',
                                       columns='particle',
                                       aggfunc='mean').values
    nofrm = len(xdf)
    noprt = len(xdf[0])
    
    ydf = pd.read_csv(general_directory + '/data_quiver.csv', usecols=[9, 10, 19])
    ydf = ydf.reset_index().pivot_table(index='frame',
                                       columns='particle',
                                       aggfunc='mean').values
    
    vdf = pd.read_csv(general_directory + '/data_quiver.csv', usecols=[9, 10, 17])
    vdf = vdf.reset_index().pivot_table(index='frame',
                                       columns='particle',
                                       aggfunc='mean').values
         
    vdf[vdf>=1e-6] = np.nan

    print('Max value of array', np.nanmax(vdf))
    vmax = np.nanmax(vdf)
    vdf = vdf/vmax
    print('\n')
    print('Normalised velocity: ',vdf)
    
    print(vdf[30,0])
    print(vdf[0,30])
    
    vxdf = pd.read_csv(general_directory + '/data_quiver.csv', usecols=[9, 10, 16])
    vxdf = vxdf.reset_index().pivot_table(index='frame',
                                       columns='particle',
                                       aggfunc='mean').values
    vxdf = vxdf/vmax

    vydf = pd.read_csv(general_directory + '/data_quiver.csv', usecols=[9, 10, 15])
    vydf = vydf.reset_index().pivot_table(index='frame',
                                       columns='particle',
                                       aggfunc='mean').values
    vydf = vydf/vmax
    
    for p in range(1, noprt-1):
        for f in range(0, nofrm-1):
            if (f%npstep == 0):
                xdf[f, p] = np.mean(xdf[f-npstep:f, p])
                ydf[f, p] = np.mean(ydf[f-npstep:f, p])
                vdf[f, p] = np.mean(vdf[f-npstep:f, p])
                vxdf[f, p] = np.mean(vxdf[f-npstep:f, p])
                vydf[f, p] = np.mean(vydf[f-npstep:f, p])
            else:
                xdf[f, p] = np.nan
                ydf[f, p] = np.nan
                vdf[f, p] = np.nan
                vxdf[f, p] = np.nan
                vydf[f, p] = np.nan
                
    for p in range(1, noprt-1): # set condition for if two particles are in the same co-ordinate
        for p2 in range(1, noprt-1):
            for f in range(0, nofrm-1, npstep):
                n = 1
                if xdf[f, p] == xdf[f, p2] and ydf[f, p] == ydf[f, p2]:
                    vdf[f, p] += vdf[f, p2] # Append p2 to value of p
                    vxdf[f, p] += vxdf[f, p2]
                    vydf[f, p] += vydf[f, p2]

                    xdf[f, p2] = np.nan # Set p2 to NaN to drop when back in dataframe
                    ydf[f, p2] = np.nan
                    vdf[f, p2] = np.nan
                    vxdf[f, p2] = np.nan
                    vydf[f, p2] = np.nan
                    n += 1

                vdf[f, p] = (vdf[f, p])/n # Average of variables in (x,y) co-ordinate
                vxdf[f, p] = (vxdf[f, p])/n
                vydf[f, p] = (vydf[f, p])/n
    
    print('Velocity vals: ',vdf)
    xs = np.linspace(0, 500, 11)
    ys = np.linspace(0,1300, 27)
    xi,yi = np.meshgrid(xs, ys)
    
    v_vals = vx_vals = vy_vals = np.array([[[0 for col in range(11)] 
                                   for row in range(27)]
                                  for x in range(nofrm)]) # 3D array of form array[f][x][yrame]
    print('3D vel shape: ',v_vals.shape,'2D vel shape: ',vdf.shape)
    for f in range(0, nofrm-1):
        for p in range(1, noprt-1):
            for x in range(0, 500, 50):
                for y in range(0, 1300, 50):
                    if ((np.isnan(vdf[f, p]) == False)
                        and ((ydf[f, p]) == y)
                        and ((xdf[f, p]) == x)):
                            v_vals[f, x, y] = vdf[f, p]
                            vx_vals[f, x, y] = vxdf[f, p]
                            vy_vals[f, x, y] = vydf[f, p]
    
    print('Preview - velocity in frames 58-62:',v_vals[58:62,:,:])
    print('shape x =',xi.shape,'shape y =',yi.shape,'shape x =',yi.shape,
         'shape v =',v_vals.shape, 'shape vx =',vx_vals.shape, 'shape vy =',vy_vals.shape)
    print('Max velocity = ',np.nanmax(v_vals))
    print('\n')
    print('All done. Now producing vector plots: ')
    print('\n')    
    
    if not os.path.exists(general_directory + "/vector_frames/"):
        os.makedirs(general_directory + "/vector_frames/")
        
    for i in range(0, nofrm-1, npstep): 
        
        fig = plt.figure(figsize=(5,13))
        ax = fig.add_subplot(1, 1, 1)
        ax.set_xlim(0, 500)
        ax.set_ylim(0, 1300)
        
        v = v_vals[i,:,:]
        vx = vx_vals[i,:,:]
        vy = vy_vals[i,:,:]
        
        quiv = ax.quiver(xi, yi, vx, vy, [v], cmap='RdBu_r', headlength=3)
        plt.title("t = " + str(i) + "s")
        
        divider = make_axes_locatable(ax)
        cax = divider.append_axes('right', size='5%', pad=0.05)
        fig.colorbar(quiv, cax=cax, orientation='vertical')
        plt.savefig(general_directory + "/vector_frames/vector_" + str(i) + ".png")
        plt.close()
        
    vecim = []
    for i in range(0, nofrm-1, npstep):
        vecim.append(imageio.imread(general_directory + "/vector_frames/vector_"
                                     + str(i) + ".png"))
    imageio.mimsave((general_directory + "/vector_frames/vector.gif"), vecim)

    print('\n')
    print('And finally heatmap plots: ')
    print('\n')    
    
    if not os.path.exists(general_directory + "/heatmap_frames/"):
        os.makedirs(general_directory + "/heatmap_frames/")

    for i in range(0, nofrm-1, npstep): 
    
        heatfig = plt.figure(figsize=(5,13))
        axh = heatfig.add_subplot(1, 1, 1)
        axh.set_xlim(0, 500)
        axh.set_ylim(0, 1300)       
        
        im = plt.contourf(xi, yi, v_vals[i,:,:], cmap='RdBu_r', vmax=1, vmin=0)
        plt.title("t = " + str(i) + "s")        
        
        divider = make_axes_locatable(axh)
        cax = divider.append_axes('right', size='5%', pad=0.05)
        fig.colorbar(im, cax=cax, orientation='vertical')
        
        plt.savefig(general_directory + "/heatmap_frames/heatmap_" + str(i) + ".png")
        plt.close()
        
    heatim = []
    for i in range(0, nofrm-1, npstep):
        heatim.append(imageio.imread(general_directory + "/heatmap_frames/heatmap_"
                                     + str(i) + ".png"))
    imageio.mimsave((general_directory + "/heatmap_frames/heatmap.gif"), heatim)
                          
for c, param in enumerate(pkl_param):
    import imageio
    from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
    print('\n')
    print('Processing data via pure numpy method: ')
    print('\n')
    df_to_numpy(data_dirs[c])
    print('\n')
    print('Done!')
                          



Processing data via pure numpy method: 




/Users/liamchalcroft/opt/miniconda3/envs/uobenv/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in greater_equal


Max value of array 4.742591904394578e-07


Normalised velocity:  [[       nan        nan        nan ...        nan        nan        nan]
 [       nan        nan        nan ...        nan        nan        nan]
 [       nan        nan        nan ...        nan        nan        nan]
 ...
 [0.01585431 0.02445682 0.03895443 ...        nan        nan        nan]
 [0.0181034         nan 0.01766916 ...        nan        nan        nan]
 [0.02515695        nan 0.01915836 ...        nan        nan        nan]]
0.03319009024136356
nan
Velocity vals:  [[       nan        nan        nan ...        nan        nan        nan]
 [       nan        nan        nan ...        nan        nan        nan]
 [       nan        nan        nan ...        nan        nan        nan]
 ...
 [0.01585431        nan        nan ...        nan        nan        nan]
 [0.0181034         nan        nan ...        nan        nan        nan]
 [0.02515695        nan 0.01915836 ...        nan        nan        nan]]
3D vel 